## Neural network analysis

- https://arxiv.org/pdf/1811.01828.pdf

https://github.com/Verisig/verisig

# Refactor boundnedness check for HPolygon #1925

In [3]:
using Revise, LazySets, BenchmarkTools

In [10]:
P = rand(HPolygon)

HPolygon{Float64}(HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([0.6159515349391558, 0.9983466409322888], 1.974736518100653), HalfSpace{Float64,Array{Float64,1}}([0.6033188947195148, 1.0995724074013542], 1.9501545526062711), HalfSpace{Float64,Array{Float64,1}}([0.35796318499727564, 1.1356103005597236], 1.5118034051414222), HalfSpace{Float64,Array{Float64,1}}([-1.8766998630881957, 0.8716420566547223], -0.47215852850328266), HalfSpace{Float64,Array{Float64,1}}([-0.38636220314186276, 0.14083680482846916], -0.06690986753953217), HalfSpace{Float64,Array{Float64,1}}([-0.16447862623133958, -0.810507596440426], 0.9908130392940367), HalfSpace{Float64,Array{Float64,1}}([0.061075810475338166, -2.8121520144125585], 3.7895289339842164), HalfSpace{Float64,Array{Float64,1}}([0.7892312673301136, -0.6233485995235735], 3.4533001233615033)])

In [11]:
isbounded(P, false)

true

In [12]:
@btime isbounded($P, false) # master

  52.567 μs (310 allocations: 20.11 KiB)


true

In [13]:
@btime LazySets.isbounded_unit_dimensions($P) # master

  3.140 μs (76 allocations: 3.63 KiB)


true

In [14]:
@btime isbounded($P, false) # this branch

  3.123 μs (76 allocations: 3.63 KiB)


true

In [20]:
using LazySets.Arrays: SingleEntryVector

function isbounded_unit_dimensions_v2(S::LazySet{N}) where {N<:Real}
    n = dim(S)
    @inbounds for i in 1:n
        d = SingleEntryVector(i, n, one(N))
        isinf(ρ(d, S)) && return false

        d = SingleEntryVector(i, n, -one(N))
        isinf(ρ(d, S)) && return false
    end
    return true
end

isbounded_unit_dimensions_v2 (generic function with 1 method)

In [23]:
function isbounded_unit_dimensions_v1(S::LazySet{N}) where {N<:Real}
    n = dim(S)
    @inbounds for i in 1:n
        for o in [one(N), -one(N)]
            d = SingleEntryVector(i, n, o)
            if ρ(d, S) == N(Inf)
                return false
            end
        end
    end
    return true
end

isbounded_unit_dimensions_v1 (generic function with 1 method)

In [22]:
@btime isbounded_unit_dimensions_v2($P)

  3.082 μs (74 allocations: 3.44 KiB)


true

In [24]:
@btime isbounded_unit_dimensions_v1($P)

  3.131 μs (76 allocations: 3.63 KiB)


true

In [25]:
(3.131 - 3.082) / 3.131

0.01564995209198337

# Symbolic M^k for interval matrices

https://github.com/JuliaReach/IntervalMatrices.jl/issues/94#issuecomment-580273358

In [82]:
# self-contained example
using BenchmarkTools
using SymEngine, RangeEnclosures, IntervalMatrices
using MacroTools: postwalk
using MultivariatePolynomials

subidx(i) = join(Char.(0x2080 .+ convert.(UInt16, digits(i)[end:-1:1])))

function symbolic_matrix(n; name="M")
    M = Matrix{SymEngine.Basic}(undef, n, n)
    for i in 1:n, j in 1:n
        M[i, j] = name * subidx(i) * subidx(j)
    end
    return M
end

# overload to avoid conflict later
function MultivariatePolynomials.subs(ex, imat; name="M")
    n = size(imat, 1) # imat assumed square
    for i in 1:n, j in 1:n
        name_ij = Symbol(name * subidx(i) * subidx(j))
        ex == name_ij && return imat[i, j]
    end
    return ex
end

Msym_eval(Msym, M, k) = [postwalk(ex -> subs(ex, M), convert(Expr, m)) for m in Msym]

function power(M, k)
    @assert size(M, 1) == size(M, 2)
    Msym = symbolic_matrix(size(M, 1))
    X = Msym_eval(Msym^k, M, k)
    return eval.(X)
end

power (generic function with 1 method)

In [83]:
M = rand(IntervalMatrix, 4)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.993268, -0.12098]  [-1.79339, 0.485787]   …  [-0.26355, 0.503887]  
 [-0.355202, 1.71295]   [-1.74568, 0.973153]      [-0.492513, 0.139944] 
 [-1.68302, -0.115787]   [0.0355031, 2.16938]     [-0.43891, 0.0136085] 
  [0.674376, 2.54803]   [-0.0569787, 2.07937]     [-0.276957, -0.159762]

In [84]:
@btime $M^10

  5.226 μs (24 allocations: 2.25 KiB)


4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.45208e+06, 1.45987e+06]  …  [-507304, 505598]
 [-1.36008e+06, 1.3783e+06]      [-475590, 476888]
 [-1.63931e+06, 1.64233e+06]     [-571773, 569209]
 [-1.30071e+06, 1.3169e+06]      [-459956, 450070]

In [38]:
@btime power($M, 11)

  1.169 s (13693012 allocations: 915.52 MiB)


4×4 Array{Interval{Float64},2}:
 [-1.68754e+06, 2.48541e+06]  …  [-1.39364e+06, 1.0238e+06] 
 [-3.39132e+06, 2.46625e+06]     [-1.46627e+06, 1.90157e+06]
 [-1.79911e+06, 3.2219e+06]      [-1.80649e+06, 1.12452e+06]
 [-2.62918e+06, 1.80662e+06]     [-1.08363e+06, 1.47422e+06]

In [40]:
@time power(M, 20)

 11.246716 seconds (106.56 M allocations: 6.953 GiB, 15.21% gc time)


4×4 Array{Interval{Float64},2}:
 [-1.68632e+13, 2.1259e+13]   …  [-1.19202e+13, 1.0023e+13] 
 [-2.90071e+13, 2.32918e+13]     [-1.38196e+13, 1.62647e+13]
 [-1.89977e+13, 2.75568e+13]     [-1.54515e+13, 1.15174e+13]
 [-2.24881e+13, 1.77235e+13]     [-1.05339e+13, 1.26094e+13]

In [43]:
s = symbolic_matrix(10)^10;

In [46]:
s[1, 1]

(M₁₁*M₁₂ + M₁₁₀*M₁₀₂ + M₁₆*M₆₂ + M₁₇*M₇₂ + M₂₂*M₁₂ + M₃₂*M₁₃ + M₄₂*M₁₄ + M₅₂*M₁₅ + M₈₂*M₁₈ + M₉₂*M₁₉)*(((M₁₀₁*M₂₁₀ + M₂₁*M₁₁ + M₂₂*M₂₁ + M₂₆*M₆₁ + M₂₇*M₇₁ + M₃₁*M₂₃ + M₄₁*M₂₄ + M₅₁*M₂₅ + M₈₁*M₂₈ + M₉₁*M₂₉)*(M₁₂*M₇₁ + M₂₂*M₇₂ + M₃₂*M₇₃ + M₄₂*M₇₄ + M₅₂*M₇₅ + M₇₁₀*M₁₀₂ + M₇₂*M₇₇ + M₇₆*M₆₂ + M₈₂*M₇₈ + M₉₂*M₇₉) + (M₁₀₁*M₃₁₀ + M₃₁*M₁₁ + M₃₁*M₃₃ + M₃₂*M₂₁ + M₃₅*M₅₁ + M₃₆*M₆₁ + M₃₇*M₇₁ + M₃₈*M₈₁ + M₄₁*M₃₄ + M₉₁*M₃₉)*(M₁₃*M₇₁ + M₂₃*M₇₂ + M₃₃*M₇₃ + M₄₃*M₇₄ + M₅₃*M₇₅ + M₇₁₀*M₁₀₃ + M₇₃*M₇₇ + M₇₆*M₆₃ + M₈₃*M₇₈ + M₉₃*M₇₉) + (M₁₀₁*M₅₁₀ + M₃₁*M₅₃ + M₄₁*M₅₄ + M₅₁*M₁₁ + M₅₂*M₂₁ + M₅₅*M₅₁ + M₅₆*M₆₁ + M₅₇*M₇₁ + M₈₁*M₅₈ + M₉₁*M₅₉)*(M₁₅*M₇₁ + M₂₅*M₇₂ + M₃₅*M₇₃ + M₄₅*M₇₄ + M₅₅*M₇₅ + M₇₁₀*M₁₀₅ + M₇₆*M₆₅ + M₇₇*M₇₅ + M₈₅*M₇₈ + M₉₅*M₇₉) + (M₁₀₁*M₈₁₀ + M₃₁*M₈₃ + M₄₁*M₈₄ + M₈₁*M₁₁ + M₈₁*M₈₈ + M₈₂*M₂₁ + M₈₅*M₅₁ + M₈₆*M₆₁ + M₈₇*M₇₁ + M₉₁*M₈₉)*(M₁₈*M₇₁ + M₂₈*M₇₂ + M₃₈*M₇₃ + M₄₈*M₇₄ + M₅₈*M₇₅ + M₇₁₀*M₁₀₈ + M₇₆*M₆₈ + M₇₈*M₇₇ + M₈₈*M₇₈ + M₉₈*M₇₉) + (M₁₀₁*M₁₁ + M₁₀₁₀*M₁₀₁ + M₁₀₂*M₂₁ + M₁₀₃*M₃₁ + M₁₀₄*M₄₁ + M₁₀₅*M₅₁ + M₁₀

In [175]:
import IntervalArithmetic
const IA = IntervalArithmetic

M = [IA.Interval(1) IA.Interval(1); IA.Interval(1) IA.Interval(1)]

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [1, 1]  [1, 1]
 [1, 1]  [1, 1]

In [176]:
power(M, 2)

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [2, 2]  [2, 2]
 [2, 2]  [2, 2]

In [177]:
M * M

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [2, 2]  [2, 2]
 [2, 2]  [2, 2]

In [178]:
M * M * M

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [4, 4]  [4, 4]
 [4, 4]  [4, 4]

In [179]:
power(M, 3)

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [4, 4]  [4, 4]
 [4, 4]  [4, 4]

In [168]:
symbolic_matrix(2)

2×2 Array{Basic,2}:
 M₁₁  M₁₂
 M₂₁  M₂₂

In [171]:
M

2×2 Array{IntervalArithmetic.Interval{Float64},2}:
 [1, 1]  [1, 1]
 [1, 1]  [1, 1]

In [174]:
Msym = symbolic_matrix(size(M, 1))
X = Msym_eval(Msym, M, 2)
#eval.(X)

10×10 Array{Expr,2}:
 :([-0.459084, 0.516727] * 1)    …  :([-1.73115, 0.755393] * 1) 
 :([-0.497811, -0.0734267] * 1)     :([-0.103678, 0.850543] * 1)
 :([-1.71571, -0.367095] * 1)       :([-0.684736, 0.615602] * 1)
 :([-1.02058, -0.264541] * 1)       :([-1.47594, -0.521112] * 1)
 :([-1.57144, 0.779662] * 1)        :([-1.79572, -1.06507] * 1) 
 :([-0.807405, 0.410766] * 1)    …  :([-1.44449, -0.68528] * 1) 
 :([0.32996, 1.36626] * 1)          :([-2.20128, 1.96994] * 1)  
 :([-0.652974, 0.662499] * 1)       :([0.293158, 0.942785] * 1) 
 :([-0.369247, -0.0909085] * 1)     :([0.108084, 2.34153] * 1)  
 :([-2.00558, -1.86023] * 1)        :([-1.53641, -0.159813] * 1)

## Using DynamicPolynomials

In [1]:
using DynamicPolynomials

In [4]:
@polyvar x y
p = 3x^2*y + x + 2y + 1
p(x => 2, y => 1)
subs(p, x => 2, y => 1)
subs(p, y => x*y^2 + 1)
q = x

x

In [10]:
@show typeof(p)
@show typeof(q)

typeof(p) = Polynomial{true,Int64}
typeof(q) = PolyVar{true}


PolyVar{true}

In [5]:
subs(q, (x, y) => (1.0, 2.0))

1.0

In [7]:
using IntervalMatrices

In [8]:
M1 = rand(IntervalMatrix, 2)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.712703, -0.097526]  [-1.85231, -0.867406]
  [0.230998, 0.513421]   [-1.11416, 1.27316]  

In [9]:
@polyvar M[1:2, 1:2]

(PolyVar{true}[M₁₋₁ M₁₋₂; M₂₋₁ M₂₋₂],)

In [11]:
x11 = M[1, 1]

M₁₋₁

In [12]:
@show typeof(x11)

typeof(x11) = PolyVar{true}


PolyVar{true}

In [15]:
vars = vec(M) # column-major

4-element Array{PolyVar{true},1}:
 M₁₋₁
 M₂₋₁
 M₁₋₂
 M₂₋₂

In [17]:
subs(x11, vars => (M1[1, 1], M1[2, 1], M1[1, 2], M1[2, 2]))

[-0.712703, -0.097526]

In [18]:
vec(M1)

4-element reshape(::IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}, 4) with eltype Interval{Float64}:
 [-0.712703, -0.097526]
  [0.230998, 0.513421] 
 [-1.85231, -0.867406] 
 [-1.11416, 1.27316]   

In [45]:
map(x -> subs(x, vars => vec(M1)), M^10)

2×2 Array{Polynomial{true,Interval{Float64}},2}:
 [-352.484, 352.895]  [-899.685, 898.902]
 [-249.157, 249.374]  [-631.895, 632.608]

In [44]:
M1^20

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-67125.3, 68540.1]  [-178530, 177656]
 [-51414.7, 50548]    [-133642, 134695]

In [48]:
(M^4)[1, 1]

M₁₋₁⁴ + 3M₁₋₁²M₂₋₁M₁₋₂ + 2M₁₋₁M₂₋₁M₁₋₂M₂₋₂ + M₂₋₁²M₁₋₂² + M₂₋₁M₁₋₂M₂₋₂²

In [51]:
Msym2 = symbolic_matrix(2)

2×2 Array{Basic,2}:
 M₁₁  M₁₂
 M₂₁  M₂₂

In [55]:
(Msym2^4)[1, 1]

(M₂₁*M₁₁ + M₂₂*M₂₁)*(M₁₁*M₁₂ + M₂₂*M₁₂) + (M₂₁*M₁₂ + M₁₁^2)^2

In [54]:
using RangeEnclosures

In [56]:
f1(a, b, c, d) = (a+b)*(c+d)
f2(a, b, c, d) = a*c + a*d + b*c + b*d

f2 (generic function with 1 method)

In [67]:
m = rand(IntervalMatrix, 2) * 100.
@show f1(vec(m)...)
@show f2(vec(m)...)

f1(vec(m)...) = [-5118.81, 274.585]
f2(vec(m)...) = [-7846.5, 4264.12]


[-7846.5, 4264.12]

In [3]:
using DynamicPolynomials

In [7]:
# this function is fast, but it is not accurate because the polynomials are expanded..
function power_poly(M::IntervalMatrix, k::Int)
    n = size(M, 1)
    @polyvar Msym[1:n, 1:n]
    vars = vec(Msym) # symbolic variables, column-major
    Msym_k = Msym^k # symbolic matrix multiplication
    Mk = map(x -> subs(x, vars => vec(M)), Msym_k) # substitution of the intervals
    return Mk
end

power_poly (generic function with 1 method)

In [8]:
using IntervalMatrices

In [15]:
M1 = rand(IntervalMatrix, 4)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.06418, 0.28004]     [0.133538, 0.184282]  …  [-1.65346, 0.620197]
 [-0.486594, 1.99605]   [-0.0623921, 1.38228]      [0.172319, 1.73795]
  [0.0286521, 2.00274]   [0.470935, 0.63278]      [-1.98736, 1.5724]  
 [-0.835215, 0.41693]    [0.343641, 1.92403]      [-1.53953, 0.752324]

In [16]:
power_poly(M1, 5)

4×4 Array{Polynomial{true,Interval{Float64}},2}:
 [-1148.85, 975.212]  [-653.279, 725.053]  …  [-1299.1, 1108.56] 
 [-1181.01, 1333.07]  [-819.645, 804.817]     [-1358.01, 1525.81]
 [-1262.22, 1300.77]  [-820.799, 820.509]     [-1405.01, 1429.12]
 [-1226.93, 1032.57]  [-666.814, 778.77]      [-1356.2, 1183.74] 

In [17]:
M1^5

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-917.559, 724.522]  [-502.431, 547.46]   …  [-1089.61, 828.373]
 [-910.449, 1074.81]  [-617.34, 635.531]      [-1056.38, 1264.53]
 [-928.45, 936.839]   [-587.213, 590.796]     [-1116.77, 1115.86]
 [-981.2, 754.851]    [-504.325, 580.29]      [-1141.95, 884.749]

In [18]:
@polyvar M[1:2, 1:2]

(PolyVar{true}[M₁₋₁ M₁₋₂; M₂₋₁ M₂₋₂],)

In [19]:
M^3

2×2 Array{Polynomial{true,Int64},2}:
 M₁₋₁³ + 2M₁₋₁M₂₋₁M₁₋₂ + M₂₋₁M₁₋₂M₂₋₂              …  M₁₋₁²M₁₋₂ + M₁₋₁M₁₋₂M₂₋₂ + M₂₋₁M₁₋₂² + M₁₋₂M₂₋₂²
 M₁₋₁²M₂₋₁ + M₁₋₁M₂₋₁M₂₋₂ + M₂₋₁²M₁₋₂ + M₂₋₁M₂₋₂²     M₁₋₁M₂₋₁M₁₋₂ + 2M₂₋₁M₁₋₂M₂₋₂ + M₂₋₂³            

In [112]:
using SymEngine

M = convert(Matrix{SymEngine.Basic}, ["M₁₁" "M₁₂"; "M₂₁" "M₂₂"])
M^3

2×2 Array{Basic,2}:
 (M₂₁*M₁₂ + M₁₁^2)*M₁₁ + (M₂₁*M₁₁ + M₂₂*M₂₁)*M₁₂  …  (M₁₁*M₁₂ + M₂₂*M₁₂)*M₁₁ + (M₂₁*M₁₂ + M₂₂^2)*M₁₂
 (M₂₁*M₁₂ + M₁₁^2)*M₂₁ + (M₂₁*M₁₁ + M₂₂*M₂₁)*M₂₂     (M₁₁*M₁₂ + M₂₂*M₁₂)*M₂₁ + (M₂₁*M₁₂ + M₂₂^2)*M₂₂

In [118]:
M = rand(IntervalMatrix, 4)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.849004, 1.50965]  [-0.546152, 2.06405]   …  [-0.55587, 0.433497]  
  [0.928679, 1.82312]  [-1.78799, -0.102206]     [-1.84175, 0.495191]  
 [-0.497742, 1.84193]   [0.350327, 1.32886]      [-0.633314, 0.034128] 
 [-0.489613, 1.40617]  [-1.3845, 1.23307]        [-0.519603, -0.489382]

In [119]:
power_poly(M, 5)

4×4 Array{Polynomial{true,Interval{Float64}},2}:
 [-1510.03, 1478.83]  [-1519.28, 1611.71]  …  [-867.099, 918.234]
 [-1305.15, 1377.49]  [-1507.4, 1304.16]      [-852.506, 755.951]
 [-1085.53, 1039.61]  [-1109.47, 1113.5]      [-621.779, 645.877]
 [-1169.95, 1186.09]  [-1245.61, 1184.96]     [-698.03, 674.458] 

In [120]:
M^5

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1078.01, 1021.95]   [-1177.75, 1312.44]   …  [-674.571, 734.126]
  [-880.018, 1009.2]   [-1291.71, 972.591]      [-718.419, 550.861]
  [-744.809, 700.458]   [-883.578, 839.827]     [-476.614, 491.062]
  [-706.803, 759.691]   [-942.293, 840.31]      [-533.817, 474.388]

In [122]:
power(M, 5)

4×4 Array{Interval{Float64},2}:
 [-1002.48, 1014.5]    [-1106.93, 1306.34]   …  [-670.278, 733.826]
  [-787.329, 1009.2]   [-1290.5, 914.789]       [-718.419, 523.977]
  [-650.697, 700.458]   [-839.796, 838.435]     [-476.614, 480.982]
  [-636.449, 756.222]   [-939.826, 837.471]     [-531.82, 467.042] 

Now we try to use a custom `enclosure` method.

In [216]:
M1 = rand(IntervalMatrix, 2)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.40078, -1.37391]     [0.0107497, 0.111812]
 [-0.762167, -0.0171388]  [0.846548, 1.26318]  

In [218]:
n = size(M1, 1)
@polyvar M[1:n, 1:n]
vars = vec(M) # symbolic variables, column-major
M_k = M^2 # symbolic matrix multiplication

first(M_k)

M₁₋₁² + M₂₋₁M₁₋₂

In [158]:
g(y...) = map(x -> subs(x, vars => y), M_k) # substitution of the intervals

g (generic function with 2 methods)

In [159]:
g(vec(M1))

4×4 Array{Polynomial{true,Interval{Float64}},2}:
 [-198.441, 205.685]  [-191.579, 177.677]  …  [-232.208, 249.958]
 [-210.237, 214.553]  [-205.626, 195.826]     [-235.144, 277.387]
 [-371.056, 360.264]  [-334.639, 364.397]     [-492.446, 386.658]
 [-185.137, 170.358]  [-180.435, 160.059]     [-208.805, 220.849]

In [160]:
enclose(g, IntervalBox(vec(M1)), :IntervalArithmetic)

4×4 Array{Polynomial{true,Interval{Float64}},2}:
 [-192.643, 201.562]  [-187.706, 170.888]  …  [-225.493, 243.125]
 [-209.948, 211.549]  [-204.051, 189.616]     [-231.594, 276.034]
 [-364.744, 354.939]  [-330.018, 359.872]     [-488.959, 375.559]
 [-182.609, 166.966]  [-174.87, 157.914]      [-205.66, 216.483] 

In [161]:
M1^5

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-114.147, 134.769]  [-164.796, 139.332]  …  [-180.581, 209.896]
 [-131.528, 144.432]  [-174.002, 163.789]     [-186.378, 240.498]
 [-261.472, 227.391]  [-277.128, 310.477]     [-429.317, 287.814]
 [-122.724, 111.873]  [-156.787, 131.159]     [-169.925, 186.545]

In [162]:
power(M1, 5)

4×4 Array{Interval{Float64},2}:
 [-109.348, 134.769]  [-164.796, 135.29]   …  [-180.581, 205.903]
 [-131.528, 138.896]  [-174.002, 158.906]     [-179.267, 240.498]
 [-255.318, 227.391]  [-270.603, 310.477]     [-429.317, 270.095]
 [-122.724, 107.632]  [-156.787, 131.159]     [-168.616, 174.126]

In [183]:
# required 
MultivariatePolynomials.coefficienttype(::Type{Interval{Float64}}) = Interval{Float64}
MultivariatePolynomials.coefficienttype(::Type{AFF{16,Float64}}) = AFF{16,Float64}
Base.zero(::Type{AFF{16,Float64}}) = 

In [186]:
AFF()

search: AFF Affine AffExpr AffineArithmetic reset_affine_index GenericAffExpr



Affine form with center `c`, affine components `γ` and error `Δ`.

Variant where Δ is an interval

---

Make an `AF` based on an interval, which is number `i` of `n` total variables.


In [166]:
h(y...) = subs(M_k[1, 1], vars => y)

h (generic function with 1 method)

In [203]:
function hh(y...)
    ans = subs(M_k[1, 1], vars => y)
    return M
end

hh (generic function with 2 methods)

In [179]:
using AffineArithmetic

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/AffineArithmetic/vTkSy.ji for AffineArithmetic [2e89c364-fad6-56cb-99bd-ebadcd2cf8d2]
└ @ Base loading.jl:1240


In [198]:
x = hh(IntervalBox(vec(M1))...)

Polynomial{true,Interval{Float64}}


[-192.643, 201.562]

In [213]:
enclose(hh, IntervalBox(vec(M1)), :AffineArithmetic)

MethodError: MethodError: no method matching zero(::Type{AFF{16,Float64}})
Closest candidates are:
  zero(!Matched::Type{Missing}) at missing.jl:82
  zero(!Matched::Type{LibGit2.GitHash}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Pkg/src/resolve/VersionWeights.jl:19
  ...

In [222]:
M_k

2×2 Array{Polynomial{true,Int64},2}:
 M₁₋₁² + M₂₋₁M₁₋₂     M₁₋₁M₁₋₂ + M₁₋₂M₂₋₂
 M₁₋₁M₂₋₁ + M₂₋₁M₂₋₂  M₂₋₁M₁₋₂ + M₂₋₂²   

In [221]:
M

2×2 Array{PolyVar{true},2}:
 M₁₋₁  M₁₋₂
 M₂₋₁  M₂₋₂

In [223]:
M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.40078, -1.37391]     [0.0107497, 0.111812]
 [-0.762167, -0.0171388]  [0.846548, 1.26318]  

In [228]:
M1 * M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [1.80243, 1.96198]   [-0.147523, 0.12647]
 [-0.939206, 1.05312]   [0.631424, 1.59544]

In [236]:
typeof(subs(M_k[1,1], vars=>(1.0, 1.0, 2.0, 3.0)).a[1])

Float64

In [242]:
enclose((y...) -> subs(M_k[1, 1], vars => y).a[1], IntervalBox(vec(M1)), :IntervalArithmetic)

[1.80243, 1.96198]

In [245]:
enclose((y...) -> subs(M_k[1, 1], vars => y).a[1], IntervalBox(vec(M1)), :IntervalOptimisation)

InterruptException: InterruptException:

In [246]:
enclose((y...) -> subs(M_k[1, 1], vars => y).a[1], IntervalBox(vec(M1)), :AffineArithmetic)

MethodError: MethodError: no method matching one(::Type{Any})
Closest candidates are:
  one(::Type{Union{Missing, T}}) where T at missing.jl:84
  one(!Matched::Type{Missing}) at missing.jl:82
  one(!Matched::BitArray{2}) at bitarray.jl:400
  ...

In [265]:
ttt(y) = y -> subs(M_k[1, 1], vars => y).a[1]
enclose(x -> ttt(x...), IntervalBox(vec(M1)), :TaylorModels)

MethodError: MethodError: no method matching (::getfield(Main, Symbol("##108#109")))(::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64})
Closest candidates are:
  #108(::Any) at In[265]:2

In [266]:
func2 = y -> subs(M_k[1, 1], vars => y).a[1]

#110 (generic function with 1 method)

In [270]:
func2(rand(5))

0.9626987464579029

In [271]:
enclose(func2, IntervalBox(vec(M1)), :TaylorModels)

MethodError: MethodError: no method matching (::getfield(Main, Symbol("##110#111")))(::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModels.TaylorModelN{4,Interval{Float64},Float64})
Closest candidates are:
  #110(::Any) at In[266]:1

In [272]:
func2(rand(4))

0.4320403902346658

In [278]:
using TaylorModels

using RangeEnclosures: zeroBox, symBox

order = 10
N = 4
dom = IntervalBox(vec(M1))
x0 = mid(dom)
set_variables(Float64, "x", order=2order, numvars=N)

4-element Array{TaylorN{Float64},1}:
  1.0 x₁ + 𝒪(‖x‖²¹)
  1.0 x₂ + 𝒪(‖x‖²¹)
  1.0 x₃ + 𝒪(‖x‖²¹)
  1.0 x₄ + 𝒪(‖x‖²¹)

In [279]:
zBoxN = zeroBox(N)
sBoxN = symBox(N)

[-1, 1] × [-1, 1] × [-1, 1] × [-1, 1]

In [280]:
x = [TaylorModelN(i, order, IntervalBox(x0), dom) for i=1:N]

4-element Array{TaylorModelN{4,Interval{Float64},Float64},1}:
    [-1.38735, -1.38734] + [1, 1] x₁ + [0, 0]
  [-0.389653, -0.389652] + [1, 1] x₂ + [0, 0]
  [0.0612808, 0.0612809] + [1, 1] x₃ + [0, 0]
      [1.05486, 1.05487] + [1, 1] x₄ + [0, 0]

In [281]:
xnorm = [normalize_taylor(xi.pol, dom - x0, true) for xi in x]

4-element Array{TaylorN{Interval{Float64}},1}:
   [-1.38735, -1.38734] + [0.0134256, 0.0134257] x₁ + 𝒪(‖x‖¹¹)
   [-0.389653, -0.389652] + [0.372513, 0.372514] x₂ + 𝒪(‖x‖¹¹)
  [0.0612808, 0.0612809] + [0.050531, 0.0505311] x₃ + 𝒪(‖x‖¹¹)
       [1.05486, 1.05487] + [0.208315, 0.208316] x₄ + 𝒪(‖x‖¹¹)

In [282]:
xnormTM = [TaylorModelN(xi_norm, 0..0, zBoxN, sBoxN) for xi_norm in xnorm]

4-element Array{TaylorModelN{4,Interval{Float64},Float64},1}:
   [-1.38735, -1.38734] + [0.0134256, 0.0134257] x₁ + [0, 0]
   [-0.389653, -0.389652] + [0.372513, 0.372514] x₂ + [0, 0]
  [0.0612808, 0.0612809] + [0.050531, 0.0505311] x₃ + [0, 0]
       [1.05486, 1.05487] + [0.208315, 0.208316] x₄ + [0, 0]

In [283]:
evaluate(f(xnormTM...), sBoxN)

MethodError: MethodError: no method matching f(::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64})
Closest candidates are:
  f(::Any) at In[132]:1

In [291]:
func2(xnormTM...)

MethodError: MethodError: no method matching (::getfield(Main, Symbol("##110#111")))(::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64}, ::TaylorModelN{4,Interval{Float64},Float64})
Closest candidates are:
  #110(::Any) at In[266]:1

In [293]:
M_k[1, 1]

M₁₋₁² + M₂₋₁M₁₋₂

In [294]:
typeof(ans)

Polynomial{true,Int64}

In [296]:
vars

4-element Array{PolyVar{true},1}:
 M₁₋₁
 M₂₋₁
 M₁₋₂
 M₂₋₂

In [297]:
xnormTM

4-element Array{TaylorModelN{4,Interval{Float64},Float64},1}:
   [-1.38735, -1.38734] + [0.0134256, 0.0134257] x₁ + [0, 0]
   [-0.389653, -0.389652] + [0.372513, 0.372514] x₂ + [0, 0]
  [0.0612808, 0.0612809] + [0.050531, 0.0505311] x₃ + [0, 0]
       [1.05486, 1.05487] + [0.208315, 0.208316] x₄ + [0, 0]

In [295]:
subs(M_k[1, 1], vars => rand(4)).a[1]

0.942222914079472

In [298]:
subs(M_k[1, 1], vars => xnormTM).a[1]

TypeError: TypeError: in typeassert, expected Term{true,Number}, got Term{true,TaylorModelN{4,Interval{Float64},Float64}}

In [244]:
enclose((y...) -> subs(M_k[1, 1], vars => y).a[1], IntervalBox(vec(M1)), :)

UndefVarError: UndefVarError: SDPA not defined

## Using lambdify

## Substitution with SymEngine

Hello,

I currently encounter a problem using the subs function. When I run the following code,

```
@vars z
subs(z^3+z^2+z,z^3,3)
```

I get 3 + z + 3^(2/3) instead of either 3 + 3^(2/3) + 3^(1/3) or 3 + z^2 + z. Is there any trick to get the latter as output ?

Thanks in advance,

In [18]:
using SymEngine

@vars z y

subs(z^3+z^2+z, z^3=>y, y=>3)

3 + z + 3^(2/3)

In [20]:
subs(z^2+z, z^2=>y, y=>3)

3 + z

In [24]:
[subs(z^i + z, z^2=>y, y=>3) for i in 1:5]

5-element Array{Basic,1}:
           2*z
         3 + z
 z + 3*sqrt(3)
         9 + z
 z + 9*sqrt(3)

In [102]:
using SymEngine, Espresso

@vars z
poly = z^3+z^2+z

z + z^2 + z^3

In [57]:
expr = convert(Expr, poly)

:(1 + z + z ^ 2 + z ^ 3)

In [92]:
using MacroTools

@capture(:(z^3 + 1), r_ + 1)

true

In [101]:
using Espresso

ex = convert(Expr, poly)
Espresso.subs(ex, Dict(:(z^3)=>3))

:(1 + z + z ^ 2 + 3)

In [13]:
@vars z
subs(z^3+z^2+z,z,cbrt(3))

6.52233339335931

In [8]:
?SymEngine.subs

```
subs
```

Substitute values into a symbolic expression.

Examples

```
@vars x y
ex = x^2 + y^2
subs(ex, x, 1) # 1 + y^2
subs(ex, (x, 1)) # 1 + y^2
subs(ex, (x, 1), (y,x)) # 1 + x^2, values are substituted left to right.
subs(ex, x=>1)  # alternate to subs(x, (x,1))
subs(ex, x=>1, y=>1) # ditto
```


In [103]:
ex = :(z = x ^ 2 * (y + x ^ 2))
g = ExGraph(ex; x=3.0, y=2.0);     # `x` and `y` are example values from which ExGraphs learns types of these vars
evaluate!(g)                       # evaluate all expressions to fill values of intermediate nodes
g

ExGraph
  ExNode{input}(x = x | 3.0)
  ExNode{input}(y = y | 2.0)
  ExNode{constant}(tmp371 = 2 | 2)
  ExNode{call}(tmp372 = x ^ tmp371 | 9.0)
  ExNode{constant}(tmp373 = 2 | 2)
  ExNode{call}(tmp374 = x ^ tmp373 | 9.0)
  ExNode{call}(tmp375 = y + tmp374 | 11.0)
  ExNode{call}(z = tmp372 * tmp375 | 99.0)


In [106]:
g

ExGraph
  ExNode{input}(x = x | 3.0)
  ExNode{input}(y = y | 2.0)
  ExNode{constant}(tmp371 = 2 | 2)
  ExNode{call}(tmp372 = x ^ tmp371 | 9.0)
  ExNode{constant}(tmp373 = 2 | 2)
  ExNode{call}(tmp374 = x ^ tmp373 | 9.0)
  ExNode{call}(tmp375 = y + tmp374 | 11.0)
  ExNode{call}(z = tmp372 * tmp375 | 99.0)


# Inconsistency in linear_map between vrep and division algorithm
#1928

https://github.com/JuliaReach/LazySets.jl/issues/1928

In [1]:
using Revise, LazySets, Polyhedra, CDDLib, LinearAlgebra
using LazySets: dim

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/LazySets/NjrGc.ji for LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1240


In [2]:
H = Hyperrectangle(low=[0, 0.5, 0], high=[1, 1, 0.]) # 3D
@show isempty(H)
@show vertices_list(H)

M = ones(2, 3)

MH = linear_map(M, H)

@show vertices_list(MH);

P = convert(HPolytope, H)
@show isempty(P)
@show vertices_list(P)

MP_div = linear_map(M, P, algorithm="division")
@show isempty(MP_div) # should give false!

MP_vrep = linear_map(M, P, algorithm="vrep")
@show isempty(MP_vrep)
@show vertices_list(MP_vrep)

isempty(H) = false
vertices_list(H) = Array{Float64,1}[[1.0, 1.0, 0.0], [0.0, 1.0, 0.0], [1.0, 0.5, 0.0], [0.0, 0.5, 0.0]]
vertices_list(MH) = Array{Float64,1}[[0.5, 0.5], [2.0, 2.0]]
isempty(P) = false
vertices_list(P) = Array{Float64,1}[[1.0, 1.0, 0.0], [0.0, 1.0, 0.0], [1.0, 0.5, 0.0], [0.0, 0.5, 0.0]]
isempty(MP_div) = true
isempty(MP_vrep) = false
vertices_list(MP_vrep) = Array{Float64,1}[[0.5, 0.5], [2.0, 2.0]]


2-element Array{Array{Float64,1},1}:
 [0.5, 0.5]
 [2.0, 2.0]

In [3]:
using CDDLib

# see the def of P and M from my previous comment
id_out = Matrix(1.0I, size(M, 1), size(M, 1))
Ax_leq_b = [Polyhedra.HalfSpace(vcat(zeros(2), c.a), c.b) for c in constraints_list(P)]
y_eq_Mx = [Polyhedra.HyperPlane(vcat(-id_out[i, :], M[i, :]), 0.0) for i in 1:size(M, 1)];

In [4]:
 Phrep = Polyhedra.hrep(y_eq_Mx, Ax_leq_b)

H-representation Polyhedra.Intersection{Float64,Array{Float64,1},Int64}:
2-element iterator of HyperPlane{Float64,Array{Float64,1}}:
 HyperPlane([-1.0, -0.0, 1.0, 1.0, 1.0], 0.0)
 HyperPlane([-0.0, -1.0, 1.0, 1.0, 1.0], 0.0),
6-element iterator of Polyhedra.HalfSpace{Float64,Array{Float64,1}}:
 HalfSpace([0.0, 0.0, 1.0, 0.0, 0.0], 1.0)
 HalfSpace([0.0, 0.0, 0.0, 1.0, 0.0], 1.0)
 HalfSpace([0.0, 0.0, 0.0, 0.0, 1.0], 0.0)
 HalfSpace([0.0, 0.0, -1.0, 0.0, 0.0], -0.0)
 HalfSpace([0.0, 0.0, 0.0, -1.0, 0.0], -0.5)
 HalfSpace([0.0, 0.0, 0.0, 0.0, -1.0], -0.0)

In [5]:
Phrep_cdd = polyhedron(Phrep, CDDLib.Library(:float))

Polyhedron CDDLib.Polyhedron{Float64}:
2-element iterator of HyperPlane{Float64,Array{Float64,1}}:
 HyperPlane([-1.0, -0.0, 1.0, 1.0, 1.0], 0.0)
 HyperPlane([-0.0, -1.0, 1.0, 1.0, 1.0], 0.0),
6-element iterator of Polyhedra.HalfSpace{Float64,Array{Float64,1}}:
 HalfSpace([0.0, 0.0, 1.0, 0.0, 0.0], 1.0)
 HalfSpace([0.0, 0.0, 0.0, 1.0, 0.0], 1.0)
 HalfSpace([0.0, 0.0, 0.0, 0.0, 1.0], 0.0)
 HalfSpace([0.0, 0.0, -1.0, 0.0, 0.0], -0.0)
 HalfSpace([0.0, 0.0, 0.0, -1.0, 0.0], -0.5)
 HalfSpace([0.0, 0.0, 0.0, 0.0, -1.0], -0.0)

In [6]:
Peli_block = Polyhedra.eliminate(Phrep_cdd, 3:5, Polyhedra.BlockElimination())

Polyhedron CDDLib.Polyhedron{Float64}:
1-element iterator of HyperPlane{Float64,Array{Float64,1}}:
 HyperPlane([1.0, -1.0], 0.0),
5-element iterator of Polyhedra.HalfSpace{Float64,Array{Float64,1}}:
 HalfSpace([-0.0, -0.0], 1.0)
 HalfSpace([1.0, -0.0], 2.0)
 HalfSpace([-1.0, -0.0], -0.5)
 HalfSpace([-0.0, -0.0], 0.5)
 HalfSpace([-0.0, -0.0], 0.0)

In [7]:
 Peli_block = Polyhedra.removeduplicates(hrep(Peli_block))

H-representation CDDInequalityMatrix{Float64,Float64}:
1-element iterator of HyperPlane{Float64,Array{Float64,1}}:
 HyperPlane([1.0, -1.0], 0.0),
3-element iterator of Polyhedra.HalfSpace{Float64,Array{Float64,1}}:
 HalfSpace([-0.0, 0.0], 1.0)
 HalfSpace([0.5, 0.5], 2.0)
 HalfSpace([-0.5, -0.5], -0.5)

### Case m > n

In [8]:
dim(H)

3

In [9]:
M = rand(5, 3)

5×3 Array{Float64,2}:
 0.489326  0.767608  0.668907
 0.126863  0.829185  0.674583
 0.202264  0.892016  0.77413 
 0.354163  0.276443  0.72368 
 0.197969  0.82449   0.337168

In [10]:
rank(M)

3

In [11]:
linear_map(M, P, algorithm="vrep")

VPolytope{Float64}(Array{Float64,1}[[1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189], [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399], [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989], [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]])

In [12]:
M

5×3 Array{Float64,2}:
 0.489326  0.767608  0.668907
 0.126863  0.829185  0.674583
 0.202264  0.892016  0.77413 
 0.354163  0.276443  0.72368 
 0.197969  0.82449   0.337168

In [13]:
m, n = size(M)
r = rank(M)
@assert m > n
@assert r == n

In [14]:
Id_ext = Matrix([Diagonal(ones(m-n)); zeros(n, m-n)])

5×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [15]:
M

5×3 Array{Float64,2}:
 0.489326  0.767608  0.668907
 0.126863  0.829185  0.674583
 0.202264  0.892016  0.77413 
 0.354163  0.276443  0.72368 
 0.197969  0.82449   0.337168

In [16]:
Mext = hcat(M, Id_ext)

5×5 Array{Float64,2}:
 0.489326  0.767608  0.668907  1.0  0.0
 0.126863  0.829185  0.674583  0.0  1.0
 0.202264  0.892016  0.77413   0.0  0.0
 0.354163  0.276443  0.72368   0.0  0.0
 0.197969  0.82449   0.337168  0.0  0.0

In [17]:
using LazySets.Arrays

In [18]:
isinvertible(Mext)

true

In [19]:
id_out = Matrix(1.0I, m-n, m-n)

# append zeros to the existing constraints, in the last m-n coordinates
cext = [LazySets.HalfSpace(vcat(c.a, zeros(m-n)), c.b) for c in constraints_list(P)]

# fix the last m-n coordinates to zero
cext = vcat(cext, [LazySets.HalfSpace(vcat(zeros(n), id_out[i, :]), 0.0) for i in 1:(m-n)],
                  [LazySets.HalfSpace(vcat(zeros(n), -id_out[i, :]), 0.0) for i in 1:(m-n)])

Pext = HPolytope(cext);

In [20]:
Mext

5×5 Array{Float64,2}:
 0.489326  0.767608  0.668907  1.0  0.0
 0.126863  0.829185  0.674583  0.0  1.0
 0.202264  0.892016  0.77413   0.0  0.0
 0.354163  0.276443  0.72368   0.0  0.0
 0.197969  0.82449   0.337168  0.0  0.0

In [21]:
Q = linear_map(Mext, Pext, algorithm="inverse")

HPolytope{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[LazySets.HalfSpace{Float64,Array{Float64,1}}([0.0, -8.881784197001252e-16, -4.881809323685728, 3.272617087111273, 4.184353695375888], 1.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-2.220446049250313e-16, 1.3877787807814457e-16, 0.23129850706982447, -0.8247581849527279, 1.2391616182409606], 1.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-4.440892098500626e-16, 0.0, 2.300764280362159, 0.09528853729520836, -2.52114598286773], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([0.0, 8.881784197001252e-16, 4.881809323685728, -3.272617087111273, -4.184353695375888], -0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([2.220446049250313e-16, -1.3877787807814457e-16, -0.23129850706982447, 0.8247581849527279, -1.2391616182409606], -0.5), LazySets.HalfSpace{Float64,Array{Float64,1}}([4.440892098500626e-16, 0.0, -2.300764280362159, -0.09528853729520836, 2.52114598286773], -0.0), LazySets.HalfSpace{Floa

In [22]:
vertices_list(Pext)

4-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 0.0]
 [1.0, 0.5, 0.0, 0.0, 0.0]
 [0.0, 0.5, 0.0, 0.0, 0.0]

In [23]:
vertices_list(Mext * Pext)

4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189] 
 [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399]  
 [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989]
 [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]

In [24]:
vertices_list(linear_map(Mext, Pext))

4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364669, 1.022459496376419]   
 [0.7676077771209944, 0.8291853417366579, 0.8920161215663114, 0.2764433059775121, 0.8244904191998401]  
 [0.8731298666203585, 0.5414555478706533, 0.6482718282283392, 0.4923848426477108, 0.610214286776499]   
 [0.38380388856049735, 0.4145926708683292, 0.44600806078315586, 0.1382216529887561, 0.4122452095999203]

In [25]:
vertices_list(linear_map(Mext, Pext, algorithm="vrep"))

4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189] 
 [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399]  
 [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989]
 [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]

In [26]:
vertices_list(linear_map(Mext, Pext, algorithm="inverse"))

4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364669, 1.022459496376419]   
 [0.7676077771209944, 0.8291853417366579, 0.8920161215663114, 0.2764433059775121, 0.8244904191998401]  
 [0.8731298666203585, 0.5414555478706533, 0.6482718282283392, 0.4923848426477108, 0.610214286776499]   
 [0.38380388856049735, 0.4145926708683292, 0.44600806078315586, 0.1382216529887561, 0.4122452095999203]

In [27]:
vertices_list(linear_map(Mext, Pext, algorithm="division"))

4-element Array{Array{Float64,1},1}:
 [1.2569337551808566, 0.9560482187389823, 1.0942798890114949, 0.6306064956364673, 1.0224594963764195]   
 [0.7676077771209947, 0.8291853417366579, 0.8920161215663116, 0.2764433059775123, 0.8244904191998405]   
 [0.8731298666203594, 0.5414555478706535, 0.6482718282283393, 0.4923848426477112, 0.6102142867764995]   
 [0.38380388856049735, 0.41459267086832896, 0.4460080607831558, 0.13822165298875616, 0.4122452095999202]

In [28]:
zeros(3) ∈ P

false

In [29]:
vertices_list(linear_map(M, P, algorithm="division")) # wrong, should give an error (M is not invertible)

4-element Array{Array{Float64,1},1}:
 [20.00923300110145, 166.60189756805653, -172.572377765032, 15.139713563727828, 0.0]                    
 [16.371047843591466, 138.8445810948705, -143.8203276739012, 12.386930327435104, 0.0]                   
 [19.625429112540953, 166.1873048971882, -173.01838582581516, 15.001491910739071, -0.41224520959991967] 
 [15.987243955030971, 138.42998842400218, -144.26633573468436, 12.248708674446348, -0.41224520959991817]

In [30]:
vertices_list(linear_map(M, P, algorithm="inverse")) # correct, gives an error

ArgumentError: ArgumentError: algorithm "inverse" requires an invertible matrix

In [31]:
vertices_list(linear_map(M, P, algorithm="vrep")) # correct

4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189] 
 [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399]  
 [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989]
 [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]

In [32]:
using BenchmarkTools

In [33]:
@btime vertices_list(linear_map($M, $P, algorithm="vrep"))

  267.738 μs (3575 allocations: 207.25 KiB)


4-element Array{Array{Float64,1},1}:
 [1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189] 
 [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399]  
 [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989]
 [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]

In [34]:
@btime linear_map($M, $P, algorithm="vrep")

  266.629 μs (3575 allocations: 207.25 KiB)


VPolytope{Float64}(Array{Float64,1}[[1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189], [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399], [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989], [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]])

In [35]:
# see discussion in LazySets #1928
# if P : Ax <= b and y = Mx, we consider the vector z = [y, x], write the equalities and the inequalities,
# and then eliminate the last x variables (there are length(x) in total) using Polyhedra.eliminate calls
# to a backend library such as CDDLib in this case:
function _linear_map_hrep_proj(M, P)
    m, n = size(M)
    @assert dim(P) == n
    d = abs(m-n) # if d ≠ 0, better use LazySets.linear_map(M, P, algorithm="inverse") 
    id_m = Matrix(1.0I, m, m)

    # extend the polytope putting the y variables first
    Ax_leq_b = [Polyhedra.HalfSpace(vcat(zeros(m), c.a), c.b) for c in constraints_list(P)]
    y_eq_Mx = [Polyhedra.HyperPlane(vcat(-id_m[i, :], M[i, :]), 0.0) for i in 1:m]
    
    Phrep = Polyhedra.hrep(y_eq_Mx, Ax_leq_b)
    Phrep_cdd = polyhedron(Phrep, CDDLib.Library(:float))
    Peli_block = Polyhedra.eliminate(Phrep_cdd, (m+1):(m+n), Polyhedra.BlockElimination())
    Peli_block = Polyhedra.removeduplicates(hrep(Peli_block))

    return HPolytope(Peli_block)
end

_linear_map_hrep_proj (generic function with 1 method)

In [36]:
# performance check
@btime _linear_map_hrep_proj($M, $P);

  264.749 μs (1873 allocations: 129.38 KiB)


In [37]:
@btime linear_map(M, P, algorithm="vrep")

  266.589 μs (3576 allocations: 207.27 KiB)


VPolytope{Float64}(Array{Float64,1}[[1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189], [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399], [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989], [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]])

In [38]:
# sanity check
v1 = tovrep(_linear_map_hrep_proj(M, P))
v2 = linear_map(M, P, algorithm="vrep")
@show v1 ⊆ v2 && v2 ⊆ v1

v1 ⊆ v2 && v2 ⊆ v1 = true


true

In [40]:
# see discussion in LazySets #1928
function _linear_map_hrep_lift(M, P)
    m, n = size(M)
    r = rank(M)
    @assert m > n
    @assert r == n
    
    # we extend the linear map to an mxm matrix with identity in the last m-n columns
    Id_ext = Matrix([Diagonal(ones(m-n)); zeros(n, m-n)])
    Mext = hcat(M, Id_ext)
    
    id_out = Matrix(1.0I, m-n, m-n)

    # append zeros to the existing constraints, in the last m-n coordinates
    cext = [LazySets.HalfSpace(vcat(c.a, zeros(m-n)), c.b) for c in constraints_list(P)]

    # now fix the last m-n coordinates to zero
    cext = vcat(cext, [LazySets.HalfSpace(vcat(zeros(n), id_out[i, :]), 0.0) for i in 1:(m-n)],
                      [LazySets.HalfSpace(vcat(zeros(n), -id_out[i, :]), 0.0) for i in 1:(m-n)])

    Pext = HPolytope(cext)

    # now Mext is invertible and we can apply the linear with inverse function
    MP = linear_map(Mext, Pext, algorithm="inverse")
    return MP
end

_linear_map_hrep_lift (generic function with 1 method)

In [41]:
# performance check
@btime _linear_map_hrep_lift($M, $P);

  12.112 μs (168 allocations: 18.81 KiB)


In [42]:
@btime linear_map(M, P, algorithm="vrep")

  266.628 μs (3576 allocations: 207.27 KiB)


VPolytope{Float64}(Array{Float64,1}[[1.2569337551808557, 0.9560482187389823, 1.0942798890114949, 0.6306064956364668, 1.0224594963764189], [0.7676077771209942, 0.8291853417366579, 0.8920161215663114, 0.276443305977512, 0.8244904191998399], [0.8731298666203586, 0.5414555478706533, 0.6482718282283392, 0.49238484264771076, 0.6102142867764989], [0.3838038885604971, 0.41459267086832896, 0.4460080607831557, 0.138221652988756, 0.41224520959991995]])

In [47]:
266/12

22.166666666666668

In [49]:
size(M)

(5, 3)

In [50]:
dim(P)

3

In [44]:
# sanity check
v1 = tovrep(_linear_map_hrep_lift(M, P))
v2 = linear_map(M, P, algorithm="vrep")
@show v1 ⊆ v2 && v2 ⊆ v1

v1 ⊆ v2 && v2 ⊆ v1 = true


true

In [45]:
_linear_map_hrep_lift(M, P)

HPolytope{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[LazySets.HalfSpace{Float64,Array{Float64,1}}([0.0, -8.881784197001252e-16, -4.881809323685728, 3.272617087111273, 4.184353695375888], 1.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-2.220446049250313e-16, 1.3877787807814457e-16, 0.23129850706982447, -0.8247581849527279, 1.2391616182409606], 1.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-4.440892098500626e-16, 0.0, 2.300764280362159, 0.09528853729520836, -2.52114598286773], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([0.0, 8.881784197001252e-16, 4.881809323685728, -3.272617087111273, -4.184353695375888], -0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([2.220446049250313e-16, -1.3877787807814457e-16, -0.23129850706982447, 0.8247581849527279, -1.2391616182409606], -0.5), LazySets.HalfSpace{Float64,Array{Float64,1}}([4.440892098500626e-16, 0.0, -2.300764280362159, -0.09528853729520836, 2.52114598286773], -0.0), LazySets.HalfSpace{Floa

In [56]:
P = rand(HPolytope, dim=4);
M = rand(8, 4);

In [59]:
@btime linear_map($M, $P, algorithm="vrep");

  468.625 ms (9302313 allocations: 548.02 MiB)


VPolytope{Float64}(Array{Float64,1}[[0.8584287284649224, 0.43610793876701215, 1.1579257323935486, 0.5294981611125507, 1.2242350877481192, 0.4858241467778547, 0.5299025780360251, 0.8167128976902374], [0.7736039108635099, 0.3740476271267728, 0.6107352777666155, 0.7205788424827382, 0.7935884586832731, 0.6750826806139948, 0.17081692960687034, 0.4989176571239061], [0.808005837676803, 0.31717751762538443, 0.6170804079319718, 0.6656944940906986, 0.8452590131982162, 0.7141409619271011, 0.24220630146153993, 0.5963406195457237], [0.6994684319703348, 0.49660159991656366, 0.5970616464609858, 0.8388535389515257, 0.6822394251122521, 0.590912848918039, 0.01697423064839776, 0.2889730800793904], [0.6191041241693319, 0.19847115435308296, 0.2437171866981122, 0.59864580434787, 0.5120528293499366, 0.6587235963392547, 0.16738792252212037, 0.432119407434251], [0.6825434141524587, 0.4756317905745095, 0.5554923517445184, 0.8240122015765001, 0.6511986843562176, 0.5897613862325941, 0.017876133854279108, 0.283148

In [60]:
@btime _linear_map_hrep_lift($M, $P);

  28.198 μs (482 allocations: 45.25 KiB)


In [61]:
468e-3 / 28e-6

16714.285714285714

In [63]:
@btime _linear_map_hrep_proj($M, $P)

  7.499 ms (35507 allocations: 3.24 MiB)


HPolytope{Float64}(LazySets.HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[LazySets.HalfSpace{Float64,Array{Float64,1}}([-0.9014815377117815, -3.192322004990111, 1.9985328609837612, 2.032371705218291, -1.0, -0.0, -0.0, -0.0], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([2.7846467949643445, 2.684257055578964, -1.763438595198389, -1.9514544549681212, -0.0, -1.0, -0.0, -0.0], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-12.19288763030646, -22.98500588209988, 10.782336880255073, 16.11984032901728, -0.0, -0.0, -1.0, -0.0], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-6.125965260669085, -13.394986552303418, 6.1854216054268205, 8.979858569378784, -0.0, -0.0, -0.0, -1.0], 0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([0.9014815377117815, 3.192322004990111, -1.9985328609837612, -2.032371705218291, 1.0, 0.0, 0.0, 0.0], -0.0), LazySets.HalfSpace{Float64,Array{Float64,1}}([-2.7846467949643445, -2.684257055578964, 1.763438595198389, 1.9514544549681212, 0.0, 1.0

In [64]:
8e-3 / 28e-6

285.7142857142857

In [108]:
P = convert(HPolytope, rand(Hyperrectangle, dim=2))
P = rand(HPolytope, dim=6)
M = rand(6,6);

In [101]:
isinvertible(M)

true

In [109]:
@btime linear_map($M, $P, algorithm="inverse");

  9.284 μs (90 allocations: 11.44 KiB)


In [110]:
@btime linear_map($M, $P, algorithm="division");

  10.505 μs (107 allocations: 10.77 KiB)


In [90]:
(2.609 - 1.830)/2.609

0.298581832119586

## Generalization of the lift

https://math.stackexchange.com/questions/2817168/can-a-non-invertible-matrix-be-extended-to-an-invertible-one

Let $M_{m\times n}$.

$$
M^e_{(m+n)\times (m+n)} =
\begin{pmatrix}
     M_{m\times n} & I_{m\times m} \\ I_{n\times n} & 0_{n\times m}
\end{pmatrix}
$$

$$
inv(M^e_{(m+n)\times (m+n)}) =
\begin{pmatrix}
     0_{n\times m} & I_{n\times n} \\ I_{m\times m} & -M_{n\times m}
\end{pmatrix}
$$

In [113]:
m, n = 2, 4
M = rand(m, n)

2×4 Array{Float64,2}:
 0.830139  0.506439  0.0784163  0.185765
 0.157628  0.472082  0.0554617  0.943656

In [115]:
Mext = [M Matrix(1.0I, m, m); Matrix(1.0I, n, n) zeros(n, m)]

6×6 Array{Float64,2}:
 0.830139  0.506439  0.0784163  0.185765  1.0  0.0
 0.157628  0.472082  0.0554617  0.943656  0.0  1.0
 1.0       0.0       0.0        0.0       0.0  0.0
 0.0       1.0       0.0        0.0       0.0  0.0
 0.0       0.0       1.0        0.0       0.0  0.0
 0.0       0.0       0.0        1.0       0.0  0.0

In [116]:
inv_Mext = [zeros(n, m) Matrix(1.0I, n, n); Matrix(1.0I, m, m) -M]

6×6 Array{Float64,2}:
 0.0  0.0   1.0        0.0        0.0         0.0     
 0.0  0.0   0.0        1.0        0.0         0.0     
 0.0  0.0   0.0        0.0        1.0         0.0     
 0.0  0.0   0.0        0.0        0.0         1.0     
 1.0  0.0  -0.830139  -0.506439  -0.0784163  -0.185765
 0.0  1.0  -0.157628  -0.472082  -0.0554617  -0.943656

In [117]:
Mext * inv_Mext

6×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [118]:
inv_Mext * Mext

6×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [244]:
# see discussion in LazySets #1928
function _linear_map_hrep_lift_v2(M, P)
    m, n = size(M)
    
    Mext = [M Matrix(1.0I, m, m); Matrix(1.0I, n, n) zeros(n, m)]
    
    id_out = Matrix(1.0I, m, m)

    # append zeros to the existing constraints, in the last m coordinates
    cext = [LazySets.HalfSpace(vcat(c.a, zeros(m)), c.b) for c in constraints_list(P)]

    # now fix the last m coordinates to zero
    cext = vcat(cext, [LazySets.HalfSpace(vcat(zeros(n), id_out[i, :]), 0.0) for i in 1:m],
                      [LazySets.HalfSpace(vcat(zeros(n), -id_out[i, :]), 0.0) for i in 1:m])

    Pext = HPolytope(cext)

    # the inverse of Mext is known and we can apply the linear with inverse function
    Mext_inv = [zeros(n, m) Matrix(1.0I, n, n); Matrix(1.0I, m, m) -M]
    MP = linear_map(Mext, Pext, inverse=Mext_inv, algorithm="inverse")
    
    return MP
    # only keep the first m coordinates
    #return HPolytope([LazySets.HalfSpace(c.a[1:m], c.b) for c in constraints_list(MP) if !iszero(c.a[1:m])])
end

_linear_map_hrep_lift_v2 (generic function with 1 method)

In [258]:
P = rand(HPolytope, dim=4);

In [259]:
M = rand(2, 4)

2×4 Array{Float64,2}:
 0.924443   0.721131  0.918632  0.917156
 0.0203537  0.709373  0.145303  0.922856

In [260]:
Pout = _linear_map_hrep_lift_v2(M, P);

In [261]:
convex_hull([v[1:2] for v in vertices_list(Pout)])

4-element Array{Array{Float64,1},1}:
 [-0.5238640966240138, -0.05152274825463088]
 [2.2443407351102795, 0.240166006770957]    
 [5.118606963490539, 2.2680092723942944]    
 [1.2720908778041924, 1.3441883967425725]   

In [262]:
vertices_list(linear_map(M, P, algorithm="vrep"))

4-element Array{Array{Float64,1},1}:
 [-0.5238640966237547, -0.05152274825452914]
 [2.2443407351103435, 0.24016600677100108]  
 [5.118606963491523, 2.268009272394667]     
 [1.2720908778045135, 1.3441883967427615]   

In [252]:
Pout.constraints[1]

LazySets.HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, -0.06311734884135688, -0.06017436160322959, 0.1864869467424986], 0.1088599839373961)

In [270]:
linear_map([1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0], Pout, algorithm="vrep")

VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-0.5238640966240138, -0.05152274825463088], [2.2443407351102795, 0.240166006770957], [5.118606963490539, 2.2680092723942944], [1.2720908778041924, 1.3441883967425725]])

In [267]:
[1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0]

2×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0

## Extension using QR

In [271]:
A = rand(5, 3)

5×3 Array{Float64,2}:
 0.710071  0.478913  0.479142
 0.330047  0.40024   0.813866
 0.582121  0.257866  0.669767
 0.927018  0.329415  0.594451
 0.566253  0.247981  0.915959

In [272]:
using LinearAlgebra

In [273]:
rank(A)

3

In [278]:
Q, R = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
5×5 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.486304  -0.408398   0.590838  -0.447177   0.218316 
 -0.226038  -0.778725  -0.31998    0.47447   -0.122411 
 -0.398675   0.111912  -0.238821  -0.372533  -0.795435 
 -0.634884   0.447518   0.218271   0.589441   0.0395771
 -0.387808   0.118329  -0.666217  -0.297806   0.550518 
R factor:
3×3 Array{Float64,2}:
 -1.46014  -0.731481  -1.41662 
  0.0      -0.301644  -0.380091
  0.0       0.0       -0.617756

In [279]:
Q * R

5×3 Array{Float64,2}:
 0.710071  0.478913  0.479142
 0.330047  0.40024   0.813866
 0.582121  0.257866  0.669767
 0.927018  0.329415  0.594451
 0.566253  0.247981  0.915959

In [282]:
Q * R

5×3 Array{Float64,2}:
 0.710071  0.478913  0.479142
 0.330047  0.40024   0.813866
 0.582121  0.257866  0.669767
 0.927018  0.329415  0.594451
 0.566253  0.247981  0.915959

In [283]:
Q

5×5 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.486304  -0.408398   0.590838  -0.447177   0.218316 
 -0.226038  -0.778725  -0.31998    0.47447   -0.122411 
 -0.398675   0.111912  -0.238821  -0.372533  -0.795435 
 -0.634884   0.447518   0.218271   0.589441   0.0395771
 -0.387808   0.118329  -0.666217  -0.297806   0.550518 

In [284]:
R

3×3 Array{Float64,2}:
 -1.46014  -0.731481  -1.41662 
  0.0      -0.301644  -0.380091
  0.0       0.0       -0.617756

In [285]:
Matrix(Q)

5×3 Array{Float64,2}:
 -0.486304  -0.408398   0.590838
 -0.226038  -0.778725  -0.31998 
 -0.398675   0.111912  -0.238821
 -0.634884   0.447518   0.218271
 -0.387808   0.118329  -0.666217

In [289]:
Qm = Matrix(Q)

5×3 Array{Float64,2}:
 -0.486304  -0.408398   0.590838
 -0.226038  -0.778725  -0.31998 
 -0.398675   0.111912  -0.238821
 -0.634884   0.447518   0.218271
 -0.387808   0.118329  -0.666217

In [292]:
A * Qm[4:5, :]'

5×2 Array{Float64,2}:
 -0.131908   -0.537914
  0.147217   -0.622846
 -0.107989   -0.641448
 -0.311378   -0.716559
 -0.0486015  -0.800481

In [293]:
a1 = A[:, 1]

5-element Array{Float64,1}:
 0.7100710794441842 
 0.33004687248954534
 0.5821205236880644 
 0.9270177313481347 
 0.5662525399951128 

In [294]:
Q * a1

5-element Array{Float64,1}:
 -0.4270812117112601
 -0.2332595384205638
 -1.1809363243695792
  0.3927824993837701
 -0.5884750849340681

In [305]:
Q' * A[:, 3]

5-element Array{Float64,1}:
 -1.4166165912323658    
 -0.3800911873872108    
 -0.6177559594241335    
  1.1102230246251565e-16
  1.1102230246251565e-16

In [308]:
Q'

5×5 Adjoint{Float64,LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}}:
 -0.486304  -0.226038  -0.398675  -0.634884   -0.387808
 -0.408398  -0.778725   0.111912   0.447518    0.118329
  0.590838  -0.31998   -0.238821   0.218271   -0.666217
 -0.447177   0.47447   -0.372533   0.589441   -0.297806
  0.218316  -0.122411  -0.795435   0.0395771   0.550518

In [309]:
Qm

5×3 Array{Float64,2}:
 -0.486304  -0.408398   0.590838
 -0.226038  -0.778725  -0.31998 
 -0.398675   0.111912  -0.238821
 -0.634884   0.447518   0.218271
 -0.387808   0.118329  -0.666217

In [310]:
Q * A

5×3 Array{Float64,2}:
 -0.427081  -0.337166  -0.235523
 -0.23326   -0.3765    -0.786469
 -1.18094   -0.527693  -1.20993 
  0.392782   0.13533    0.592856
 -0.588475  -0.271745  -0.208501

In [312]:
A * Q'

5×5 Array{Float64,2}:
 -0.257803    -0.686761  -0.343921  -0.131908   -0.537914
  0.156902    -0.646701  -0.281158   0.147217   -0.622846
  0.00732392  -0.5467    -0.363173  -0.107989   -0.641448
 -0.234121    -0.656278  -0.474681  -0.311378   -0.716559
  0.164537    -0.614192  -0.416749  -0.0486015  -0.800481

In [313]:
Q' * A

5×3 Array{Float64,2}:
 -1.46014      -0.731481     -1.41662    
  5.55112e-17  -0.301644     -0.380091   
 -5.55112e-16  -1.11022e-16  -0.617756   
  1.11022e-16   0.0           1.11022e-16
 -3.33067e-16  -5.55112e-17   1.11022e-16

In [316]:
Matrix(Q') * A

5×3 Array{Float64,2}:
 -1.46014      -0.731481     -1.41662    
  4.63162e-17  -0.301644     -0.380091   
 -2.85432e-16  -1.2122e-16   -0.617756   
  1.67941e-16   2.67891e-17   1.32073e-16
 -1.45078e-17  -2.73795e-17   4.75897e-17

In [319]:
q1 = Matrix(Q')[4, :]

5-element Array{Float64,1}:
 -0.4471770565152565 
  0.47447031404774787
 -0.3725329301877762 
  0.5894414579132832 
 -0.29780628742952225

In [325]:
R

3×3 Array{Float64,2}:
 -1.46014  -0.731481  -1.41662 
  0.0      -0.301644  -0.380091
  0.0       0.0       -0.617756

In [322]:
A

5×3 Array{Float64,2}:
 0.710071  0.478913  0.479142
 0.330047  0.40024   0.813866
 0.582121  0.257866  0.669767
 0.927018  0.329415  0.594451
 0.566253  0.247981  0.915959

In [324]:
q1' * A

1×3 Adjoint{Float64,Array{Float64,1}}:
 1.11022e-16  4.16334e-17  5.55112e-17

In [327]:
size(Q)

(5, 5)

In [399]:
function extend(M)
    Q, R = qr(M)
    a = iszero(R) ? 1 : size(R, 1)+1
    b = size(Q, 1)
    ext = Matrix(Q')[a:b, :]
    Mext = hcat(M, ext')
    return Mext
end

extend (generic function with 1 method)

In [460]:
m = rand(5,3)
M = hcat(m, 2*m[:, 1])

5×4 Array{Float64,2}:
 0.18459   0.990444  0.514058   0.36918 
 0.346679  0.899818  0.0919997  0.693359
 0.838223  0.233413  0.436089   1.67645 
 0.728239  0.979405  0.640922   1.45648 
 0.326964  0.190715  0.337279   0.653929

In [461]:
rank(M)

3

In [462]:
Mext = extend(M)

5×5 Array{Float64,2}:
 0.18459   0.990444  0.514058   0.36918   -0.33183  
 0.346679  0.899818  0.0919997  0.693359   0.212451 
 0.838223  0.233413  0.436089   1.67645   -0.401772 
 0.728239  0.979405  0.640922   1.45648    0.0758447
 0.326964  0.190715  0.337279   0.653929   0.823153 

In [463]:
rank(Mext)

4

In [469]:
using SparseArrays

In [474]:
M = sprandn(6, 1, 0.3);

In [475]:
rank(M)

1

In [476]:
rank(extend(Matrix(M)))

6

In [477]:
Q, R = qr(Matrix(M))

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
6×6 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 0.0        0.397433   0.0   0.903805   0.158692   0.0
 0.397433   0.842047   0.0  -0.359202  -0.0630695  0.0
 0.0        0.0        1.0   0.0        0.0        0.0
 0.903805  -0.359202   0.0   0.183136  -0.143427   0.0
 0.158692  -0.0630695  0.0  -0.143427   0.974817   0.0
 0.0        0.0        0.0   0.0        0.0        1.0
R factor:
1×1 Array{Float64,2}:
 -1.6181299591837748

In [494]:
M = rand(5, 2)
Q, R = qr(M)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
5×5 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.763954   0.257102  -0.28777    -0.304602   -0.417945 
 -0.258458  -0.780809   0.0199516  -0.46947     0.320527 
 -0.27016    0.101493   0.953805   -0.0321398  -0.0770511
 -0.425049  -0.362887  -0.0548304   0.82735    -0.0115205
 -0.309711   0.426906  -0.0635664   0.0357078   0.846472 
R factor:
2×2 Array{Float64,2}:
 -1.30566  -1.19759 
  0.0      -0.772038

In [495]:
a = size(R, 2) + 1

3

In [496]:
b = size(Q, 1)

5

In [497]:
Matrix(Q')

5×5 Array{Float64,2}:
 -0.763954  -0.258458   -0.27016    -0.425049   -0.309711 
  0.257102  -0.780809    0.101493   -0.362887    0.426906 
 -0.28777    0.0199516   0.953805   -0.0548304  -0.0635664
 -0.304602  -0.46947    -0.0321398   0.82735     0.0357078
 -0.417945   0.320527   -0.0770511  -0.0115205   0.846472 

In [646]:
M = rand(7, 4)
@show rank(M)
m, n = size(M)
Q, R = qr(M)
Q2 = Matrix(Q')[(n+1):m, :]
Mext = hcat(M, Q2')
@show rank(Mext)
inv_Mext = vcat(inv(R) * Q', Q2)

rank(M) = 4
rank(Mext) = 7


7×7 Array{Float64,2}:
  0.157115  -0.829711    0.176625    …  -0.517393   0.611982   0.409179 
 -0.128649   0.971191   -1.63917         0.263766   0.350514  -0.161872 
 -0.207224   0.768506   -0.177402       -0.330586  -0.385159   0.790722 
  0.314955  -0.288246    1.27072         0.780566  -0.363338  -0.715371 
 -0.598788  -0.413799    0.00409532      0.534236  -0.187058   0.280156 
 -0.598358   0.356505    0.233497    …  -0.022523   0.632264  -0.0654498
  0.221328  -0.0021093  -0.397409        0.390497   0.182227   0.45555  

In [647]:
M

7×4 Array{Float64,2}:
 0.866409    0.574404  0.491696  0.870269
 0.00584593  0.339578  0.664685  0.465188
 0.772301    0.256527  0.758439  0.949192
 0.909763    0.724919  0.593944  0.736363
 0.471371    0.656251  0.336002  0.952464
 0.963311    0.568444  0.138212  0.56008 
 0.726248    0.162609  0.906965  0.389025

In [648]:
inv(Mext)

7×7 Array{Float64,2}:
  0.157115  -0.829711    0.176625    …  -0.517393   0.611982   0.409179 
 -0.128649   0.971191   -1.63917         0.263766   0.350514  -0.161872 
 -0.207224   0.768506   -0.177402       -0.330586  -0.385159   0.790722 
  0.314955  -0.288246    1.27072         0.780566  -0.363338  -0.715371 
 -0.598788  -0.413799    0.00409532      0.534236  -0.187058   0.280156 
 -0.598358   0.356505    0.233497    …  -0.022523   0.632264  -0.0654498
  0.221328  -0.0021093  -0.397409        0.390497   0.182227   0.45555  

In [649]:
inv_Mext

7×7 Array{Float64,2}:
  0.157115  -0.829711    0.176625    …  -0.517393   0.611982   0.409179 
 -0.128649   0.971191   -1.63917         0.263766   0.350514  -0.161872 
 -0.207224   0.768506   -0.177402       -0.330586  -0.385159   0.790722 
  0.314955  -0.288246    1.27072         0.780566  -0.363338  -0.715371 
 -0.598788  -0.413799    0.00409532      0.534236  -0.187058   0.280156 
 -0.598358   0.356505    0.233497    …  -0.022523   0.632264  -0.0654498
  0.221328  -0.0021093  -0.397409        0.390497   0.182227   0.45555  

In [650]:
opnorm(Mext * inv_Mext - I)

9.561668562259607e-16

In [651]:
opnorm(inv_Mext * Mext - I)

8.61033711578544e-16

In [663]:
using LinearAlgebra

# we assume that size(M, 1) >= size(M, 2)
# we assume that rank(M) = size(M, 2)
# it holds that Q2 * M = 0; Q2 is the null space of M
# this function also returns the inverse of the extended matrix Mext,
# because it is readily available from the QR decomposition
function _extend_orthogonal(M)
    m, n = size(M)
    Q, R = qr(M)
    Q2 = Matrix(Q')[(n+1):m, :]
    Mext = hcat(M, Q2')
    inv_Mext = vcat(inv(R) * Q', Q2)
    return Mext, inv_Mext
end

# final version. the check can be made optional..
function extend2(M)
    m, n = size(M)
    @assert rank(M) == n && m > n

    Q, R = qr(M)
    Q2 = Q[:, (n + 1):end]
    Mext = hcat(M, Q2)
    inv_Mext = vcat(inv(R) * Q', Q2')
    return Mext, inv_Mext
end

function _linear_map_hrep_lift_v3(M, P)
    m, n = size(M)
    r = rank(M)
    @assert m > n
    @assert r == n
    
    # we extend M to an invertible m x m matrix by appending m-n columns
    # orthogonal to the column space of M
    Mext, inv_Mext = _extend_orthogonal(M)

    # append zeros to the existing constraints, in the last m-n coordinates
    cext = [LazySets.HalfSpace(vcat(c.a, zeros(m-n)), c.b) for c in constraints_list(P)]

    # now fix the last m-n coordinates to zero
    id_out = Matrix(1.0I, m-n, m-n)
    cext = vcat(cext, [LazySets.HalfSpace(vcat(zeros(n), id_out[i, :]), 0.0) for i in 1:(m-n)],
                      [LazySets.HalfSpace(vcat(zeros(n), -id_out[i, :]), 0.0) for i in 1:(m-n)])

    Pext = HPolytope(cext)

    # now Mext is invertible and we can apply the linear with inverse function
    MP = linear_map(Mext, Pext, algorithm="inverse")
    return MP
end

_linear_map_hrep_lift_v3 (generic function with 1 method)

In [671]:
M  = rand(5, 3)

5×3 Array{Float64,2}:
 0.872137   0.728485  0.617654 
 0.0607614  0.848517  0.0209029
 0.93249    0.946171  0.282096 
 0.861927   0.708566  0.472554 
 0.560566   0.141609  0.116406 

In [677]:
@btime Mext, inv_Mext = _extend_orthogonal($M);

  21.532 μs (96 allocations: 11.53 KiB)


In [678]:
@btime Mext2, inv_Mext2 = extend2($M);

  13.343 μs (57 allocations: 8.30 KiB)


In [674]:
Mext == Mext2

true

In [675]:
inv_Mext == inv_Mext2

true

5×5 Array{Float64,2}:
  1.0          -2.1474e-16   -1.55092e-16   3.75872e-17   2.22793e-17
  3.50602e-17   1.0          -2.61602e-17   8.07603e-18   4.26609e-17
 -3.14687e-16  -1.64938e-16   1.0          -1.09674e-16  -7.20909e-17
 -2.39969e-16  -1.30702e-16  -5.41625e-17   1.0          -1.12914e-16
 -6.09874e-17  -1.28503e-17  -1.65798e-17  -2.96476e-17   1.0        

# Advance with RA

In [1]:
using Revise, ReachabilityAnalysis

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/ReachabilityAnalysis/fYI0z.ji for ReachabilityAnalysis [1e97bd63-91d1-579d-8e8d-501d2b57c93f]
└ @ Base loading.jl:1240


In [3]:
A = 
S = @system(x' = Ax)
P = InitialValueProblem(S, [1.0, 0.0] ⊕ BallInf(zeros(2), 0.001))
# P = @ivp(x' = Ax, x ∈ [1.0, 0.0] ⊕ BallInf(zeros(2), 0.001))
# function rotation!(dx, x, p, t)
#     dx[1] = x[2]
#     dx[2] = -x[1]
# end
# @ivp(x' = rotation!, x ∈ [1.0, 0.0] ⊕ BallInf(zeros(2), 0.001))

InitialValueProblem{LinearContinuousSystem{Float64,Array{Float64,2}},Translation{Float64,Array{Float64,1},BallInf{Float64}}}(LinearContinuousSystem{Float64,Array{Float64,2}}([0.0 1.0; -1.0 0.0]), Translation{Float64,Array{Float64,1},BallInf{Float64}}(BallInf{Float64}([0.0, 0.0], 0.001), [1.0, 0.0]))

In [ ]:
solve(P)